In [135]:
import numpy as np
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from astropy.table import Table
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

We will train a model to automatically classify galaxies by morphology (labeled data from SDSS and Galaxy Zoo). 
We follow Banjeri et al 2010, MNRAS 406, 342-353

In [136]:
file = 'data/Skyserver_galaxymorph2.fit'
data = Table.read(file, format='fits')
df = data.to_pandas()
df.head()

,dered_g,dered_r,dered_i,deVAB_i,expAB_i,lnLexp_i,lnLdeV_i,lnLstar_i,nvote,p_el,p_cw,p_acw,p_edge,p_dk,p_mg,p_cs
0,16.157143,15.339664,14.977295,0.771919,0.731467,-9126.769531,-393.420349,-42334.523438,21,1.000,0.000,0.000,0.000,0.000,0.000,0.000
1,17.727175,17.277601,17.138424,0.718036,0.752297,-0.122561,-22.863409,-321.968689,30,0.167,0.133,0.433,0.167,0.100,0.000,0.733
2,17.465136,17.251240,17.083557,0.788438,0.828909,-59.886116,-60.950695,-1870.144775,30,0.400,0.000,0.133,0.267,0.167,0.033,0.400
3,16.852707,16.538921,16.299740,0.292246,0.219601,-14.584803,-435.035919,-2757.247070,33,0.061,0.000,0.000,0.909,0.000,0.030,0.909
4,17.990713,17.596178,17.506077,0.193646,0.842761,-100.243294,-71.750786,-104.684471,66,0.364,0.000,0.015,0.030,0.348,0.242,0.045


In [137]:
# fix random seed for reprodutable = Table.read(data)
seed = 7
numpy.random.seed(seed)

In [138]:
#df['g-r', 'r-i'] = df['dered_g']-df['dered_r'], df['dered_r']-df['dered_i']
df['g-r']= df['dered_g']-df['dered_r']
df['r-i']= df['dered_r']-df['dered_i']
df.head()

,dered_g,dered_r,dered_i,deVAB_i,expAB_i,lnLexp_i,lnLdeV_i,lnLstar_i,nvote,p_el,p_cw,p_acw,p_edge,p_dk,p_mg,p_cs,g-r,r-i
0,16.157143,15.339664,14.977295,0.771919,0.731467,-9126.769531,-393.420349,-42334.523438,21,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.817478,0.362370
1,17.727175,17.277601,17.138424,0.718036,0.752297,-0.122561,-22.863409,-321.968689,30,0.167,0.133,0.433,0.167,0.100,0.000,0.733,0.449574,0.139177
2,17.465136,17.251240,17.083557,0.788438,0.828909,-59.886116,-60.950695,-1870.144775,30,0.400,0.000,0.133,0.267,0.167,0.033,0.400,0.213896,0.167683
3,16.852707,16.538921,16.299740,0.292246,0.219601,-14.584803,-435.035919,-2757.247070,33,0.061,0.000,0.000,0.909,0.000,0.030,0.909,0.313786,0.239182
4,17.990713,17.596178,17.506077,0.193646,0.842761,-100.243294,-71.750786,-104.684471,66,0.364,0.000,0.015,0.030,0.348,0.242,0.045,0.394535,0.090101


In [139]:
#predictors = ['modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i']
predictors = ['g-r', 'r-i', 'deVAB_i', 'expAB_i', 'lnLexp_i', 'lnLdeV_i', 'lnLstar_i']
types = ['p_el', 'p_cs', 'p_acw', 'p_edge', 'p_dk', 'p_mg', 'p_cs']
df['class'] = DataFrame.idxmax(df[types], axis=1)
df.head()

,dered_g,dered_r,dered_i,deVAB_i,expAB_i,lnLexp_i,lnLdeV_i,lnLstar_i,nvote,p_el,p_cw,p_acw,p_edge,p_dk,p_mg,p_cs,g-r,r-i,class
0,16.157143,15.339664,14.977295,0.771919,0.731467,-9126.769531,-393.420349,-42334.523438,21,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.817478,0.362370,p_el
1,17.727175,17.277601,17.138424,0.718036,0.752297,-0.122561,-22.863409,-321.968689,30,0.167,0.133,0.433,0.167,0.100,0.000,0.733,0.449574,0.139177,p_cs
2,17.465136,17.251240,17.083557,0.788438,0.828909,-59.886116,-60.950695,-1870.144775,30,0.400,0.000,0.133,0.267,0.167,0.033,0.400,0.213896,0.167683,p_el
3,16.852707,16.538921,16.299740,0.292246,0.219601,-14.584803,-435.035919,-2757.247070,33,0.061,0.000,0.000,0.909,0.000,0.030,0.909,0.313786,0.239182,p_cs
4,17.990713,17.596178,17.506077,0.193646,0.842761,-100.243294,-71.750786,-104.684471,66,0.364,0.000,0.015,0.030,0.348,0.242,0.045,0.394535,0.090101,p_el


In [140]:
list(enumerate(types))

[(0, 'p_el'),
 (1, 'p_cs'),
 (2, 'p_acw'),
 (3, 'p_edge'),
 (4, 'p_dk'),
 (5, 'p_mg'),
 (6, 'p_cs')]

In [141]:
#predictors = ['modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i']

In [142]:
df['response'] = 1.
for i in range(len(types)):
    response = types[i]
    print(response)
    msk = df['class'] == response
    df['response'][msk] = i

df[['class', 'response']]   

p_el
p_cs
p_acw


/Users/ceciliagarraffo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


p_edge
p_dk
p_mg
p_cs


,class,response
0,p_el,0.0
1,p_cs,6.0
2,p_el,0.0
3,p_cs,6.0
4,p_el,0.0
5,p_el,0.0
6,p_el,0.0
7,p_el,0.0
8,p_el,0.0
9,p_dk,4.0


In [143]:
#df['modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'response'].head()

In [144]:
df.head()

,dered_g,dered_r,dered_i,deVAB_i,expAB_i,lnLexp_i,lnLdeV_i,lnLstar_i,nvote,p_el,p_cw,p_acw,p_edge,p_dk,p_mg,p_cs,g-r,r-i,class,response
0,16.157143,15.339664,14.977295,0.771919,0.731467,-9126.769531,-393.420349,-42334.523438,21,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.817478,0.362370,p_el,0.0
1,17.727175,17.277601,17.138424,0.718036,0.752297,-0.122561,-22.863409,-321.968689,30,0.167,0.133,0.433,0.167,0.100,0.000,0.733,0.449574,0.139177,p_cs,6.0
2,17.465136,17.251240,17.083557,0.788438,0.828909,-59.886116,-60.950695,-1870.144775,30,0.400,0.000,0.133,0.267,0.167,0.033,0.400,0.213896,0.167683,p_el,0.0
3,16.852707,16.538921,16.299740,0.292246,0.219601,-14.584803,-435.035919,-2757.247070,33,0.061,0.000,0.000,0.909,0.000,0.030,0.909,0.313786,0.239182,p_cs,6.0
4,17.990713,17.596178,17.506077,0.193646,0.842761,-100.243294,-71.750786,-104.684471,66,0.364,0.000,0.015,0.030,0.348,0.242,0.045,0.394535,0.090101,p_el,0.0


Split predictors and response

In [145]:
print(predictors[:], ['response'])
df[predictors[:]]

['g-r', 'r-i', 'deVAB_i', 'expAB_i', 'lnLexp_i', 'lnLdeV_i', 'lnLstar_i'] ['response']


,g-r,r-i,deVAB_i,expAB_i,lnLexp_i,lnLdeV_i,lnLstar_i
0,0.817478,0.362370,0.771919,0.731467,-9126.769531,-393.420349,-42334.523438
1,0.449574,0.139177,0.718036,0.752297,-0.122561,-22.863409,-321.968689
2,0.213896,0.167683,0.788438,0.828909,-59.886116,-60.950695,-1870.144775
3,0.313786,0.239182,0.292246,0.219601,-14.584803,-435.035919,-2757.247070
4,0.394535,0.090101,0.193646,0.842761,-100.243294,-71.750786,-104.684471
5,0.918608,0.424013,0.621988,0.614841,-1319.880249,-311.409271,-15937.595703
6,0.218086,0.296398,0.470560,0.529419,-205.487717,-4.703940,-1604.308716
7,0.596766,0.414845,0.983522,0.974777,-114.888725,-14.876837,-2082.090576
8,0.366402,0.192118,0.582849,0.528655,-2853.558594,-1013.533142,-3383.770996
9,0.366399,0.089875,0.050000,0.076823,-212.310791,-874.054810,-5185.847656


In [146]:
df['response'][:10]

0    0.0
1    6.0
2    0.0
3    6.0
4    0.0
5    0.0
6    0.0
7    0.0
8    0.0
9    4.0
Name: response, dtype: float64

In [147]:
select = np.append(predictors, 'response')
msk = np.random.random(len(df['response'])) >= 0.99
dataset = df[select][msk].values
print(dataset.shape)
X = dataset[:, 0:-1].astype(float)
Y = dataset[:,-1]
print(Y)

(629, 8)
[0. 0. 6. 6. 6. 0. 5. 4. 4. 5. 6. 6. 5. 0. 0. 0. 0. 4. 4. 0. 0. 0. 4. 0.
 4. 0. 4. 4. 0. 6. 4. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 4. 6. 6. 0. 6. 6.
 6. 0. 0. 0. 6. 4. 0. 6. 4. 0. 6. 0. 0. 6. 6. 0. 0. 0. 6. 6. 0. 6. 4. 6.
 6. 4. 6. 0. 6. 4. 0. 0. 6. 0. 6. 0. 0. 0. 4. 6. 4. 4. 0. 0. 6. 0. 0. 0.
 0. 0. 6. 6. 0. 6. 4. 6. 6. 0. 0. 0. 6. 6. 0. 0. 0. 6. 0. 6. 0. 6. 0. 4.
 6. 0. 6. 0. 6. 0. 6. 0. 6. 6. 0. 0. 6. 0. 6. 4. 6. 6. 0. 0. 4. 0. 0. 0.
 6. 0. 6. 0. 6. 4. 0. 0. 6. 0. 0. 6. 0. 6. 6. 0. 0. 0. 6. 6. 0. 6. 0. 6.
 0. 5. 0. 0. 0. 0. 4. 0. 6. 0. 0. 0. 4. 0. 6. 6. 0. 6. 0. 6. 0. 0. 6. 0.
 0. 6. 6. 6. 0. 6. 0. 6. 6. 4. 0. 4. 6. 0. 6. 0. 0. 0. 0. 0. 6. 6. 0. 6.
 6. 0. 0. 0. 0. 6. 6. 6. 6. 0. 6. 6. 6. 5. 4. 6. 6. 0. 0. 0. 0. 6. 4. 0.
 6. 0. 6. 6. 0. 6. 6. 0. 6. 4. 0. 4. 6. 0. 0. 6. 6. 6. 6. 0. 6. 6. 0. 0.
 6. 4. 5. 0. 0. 4. 6. 6. 6. 4. 0. 6. 0. 6. 5. 0. 0. 4. 4. 6. 6. 6. 6. 0.
 0. 6. 6. 6. 6. 0. 0. 6. 0. 0. 0. 0. 0. 6. 0. 0. 4. 0. 0. 4. 0. 4. 0. 0.
 6. 6. 5. 6. 6. 0. 0. 6. 0. 6. 6. 6. 0. 0.

One Hot Encoding of The Classes

In [148]:
inp = X.shape[1] #number of inputs

In [149]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
print(dummy_y.shape[1])

4


Define neural network: 4 inputs -> [8 hidden nodes] -> 4 outputs

In [160]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=inp, activation='relu')) 
    model.add(Dense(8,  activation='relu')) # Do we need a kernel_initializer? ('uniform' or 'normal'?)
    model.add(Dense(dummy_y.shape[1], activation='softmax')) #because it is cathegorical 
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
    return model

In [161]:
#estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0) # verbose = 0 means de\bugging is turned off
#kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
#results = cross_val_score(estimator, X, dummy_y, cv=kfold)
#print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [162]:
# evaluate baseline model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=baseline_model, epochs=100,
    batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 66.68% (5.77%)


Clean data. We only want to keep p_el (elliptical), p_cw+p_acw+p_edge (spiral), and p_dk (¨don´t know¨ or point artifacts). We also keep only the ones with 98% certainly on each category.